In [248]:
import pandas as pd
import json
import fileinput
import re
import time
from bs4 import BeautifulSoup as Soup
from selenium import webdriver
import time
import re
from tqdm import tqdm

from selenium.webdriver.support.ui import Select

In [91]:
df = pd.DataFrame()
with fileinput.input(files='/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/github/Untitled/worldfootball_crawler/player_worldfootball.jl') as file:
    for line in file:
        conv = json.loads(line)
        df = df.append(conv, ignore_index=True)
df.head()

,name,player_url,season,team,team_url,DateOfBirth,height,Position
0,David Carney,https://www.worldfootball.net/player_summary/d...,2000/2001,Everton FC,https://www.worldfootball.net/teams/everton-fc/,30/11/1983,181 cm,MF
1,Stephen Carr,https://www.worldfootball.net/player_summary/s...,2000/2001,Tottenham Hotspur,https://www.worldfootball.net/teams/tottenham-...,29/08/1976,175 cm,DF
2,Jamie Carragher,https://www.worldfootball.net/player_summary/j...,2000/2001,Liverpool FC,https://www.worldfootball.net/teams/liverpool-fc/,28/01/1978,185 cm,DF
3,Michael Carrick,https://www.worldfootball.net/player_summary/m...,2000/2001,West Ham United,https://www.worldfootball.net/teams/west-ham-u...,28/07/1981,186 cm,MF
4,Lee Carsley,https://www.worldfootball.net/player_summary/l...,2000/2001,Coventry City,https://www.worldfootball.net/teams/coventry-c...,28/02/1974,178 cm,MF


In [256]:
names = list(df["name"].drop_duplicates())

In [257]:
name_changed = [re.sub(" ", "+" ,name) for name in names]

In [251]:
names

['David+Carney',
 'Stephen+Carr',
 'Jamie+Carragher',
 'Michael+Carrick',
 'Lee+Carsley',
 'David+James',
 'Tim+Flowers',
 'Patrice+Carteron',
 'Pierluigi+Casiraghi',
 'Luke+Chadwick',
 'Gary+Charles',
 'Laurent+Charvet',
 'Francis+Jeffers',
 'Claus+Jensen',
 'Eoin+Jess',
 'Phil+Jevons',
 'Julian+Joachim',
 'Joseph-Désiré+Job',
 'Terrell+Forbes',
 'Craig+Forrest',
 'Jonathan+Fortune',
 'Quinton+Fortune',
 'Youssef+Chippo',
 'Malcolm+Christie',
 'Jamie+Clapham',
 'Ben+Clark',
 'Peter+Clarke',
 'Michael+Clegg',
 'Jonatan+Johansson',
 'Ronny+Johnsen',
 'Seth+Johnson',
 'Slaviša+Jokanović',
 'Ben+Jones',
 'Robert+Betts',
 'Lee+Fowler',
 'Robbie+Fowler',
 'Hayden+Foxe',
 'Steffen+Freund',
 'Alexander+Cleland',
 'Stephen+Clemence',
 'Andy+Cole',
 'Ashley+Cole',
 'Joe+Cole',
 'Steve+Cooke',
 'Terry+Cooke',
 'Brad+Jones',
 'Matthew+Jones',
 'Paul+Jones',
 'Jon+Jönsson',
 'Laurent+Delorge',
 'Scott+Bevan',
 'Jonathan+Bewers',
 'Igor+Bišćan',
 'Robbie+Blake',
 'Sean+Friars',
 'Steve+Froggatt',
 

In [252]:
total_search_link = ['https://www.whoscored.com/Search/?t=' + name for name in names ]

In [253]:
len(total_search_link)

4337

In [254]:
total_player_link = []

In [281]:
%time

names = list(df["name"].drop_duplicates())
name_changed = [re.sub(" ", "+" ,name) for name in names]
total_search_link = ['https://www.whoscored.com/Search/?t=' + name for name in name_changed ]

with open('player_info_whoscored.jl', 'a') as f:
    start_time = time.time()
    driver = webdriver.Chrome(executable_path='/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/Untitled Folder/chromedriver')

    for name, url in tqdm(zip(names, total_search_link)):
        time.sleep(1)
        driver.get(url)
        page = Soup(driver.page_source, features='html.parser')
        
        links = []
        try:
            for link in page.table.find_all('a'):
                links.append(link.get('href'))
            links = ["https://www.whoscored.com" + i for i in links if "Players" in i]
        except:
            continue

        for link in links:
            res_dict = dict()
            res_dict["player_block"] = name
            res_dict["player_url"] = link
            driver.get(link)

            page = Soup(driver.page_source, features='html.parser')
            ## summary
            table = page.find('div', 'col12-lg-10 col12-m-10 col12-s-9 col12-xs-8')

            tmp = [i.split(":") for i in table.text.split("\n") if i != "" and i !=  '        ']

            flatten_list = [j.strip() for sub in tmp for j in sub]

            flatten_list = [i for i in flatten_list if i != ""]

            key_list = ["Name", "Current Team", "Shirt Number", "Age", "Height", "Nationality", "Positions"]
            for i in flatten_list:
                if i in key_list:
                    key = i
                elif i != '':
                    res_dict[key] = i


            ## weak strong
            
            table = page.find("div", "sws-content character-card singular")
            try:
                character = table.text.split("\n\n")
            except:
                f.write(json.dumps(res_dict) +'\n')
                continue

            strong = []
            weak = []
            play_style = []
            for i in character:
                if ".com" not in i and "Style of Play" not in i and "Strengths" not in i and "Weaknesses" not in i and i != "":
                    if "Strong" in i:
                        strong.append(i.strip())
                    elif "Weak" in i:
                        weak.append(i.strip())
                    else:
                        play_style.append(i.strip())


            strong = [i.split("\n")[0] for i in strong]

            weak = [i.split("\n")[0] for i in weak]

            res_dict["Strengths"] = strong
            res_dict["Weaknesses"] = weak
            res_dict["Style_of_Play"] = play_style
            
            f.write(json.dumps(res_dict) +'\n')
            

4337it [5:27:58,  4.54s/it]


## Game crawl

In [104]:
start_time = time.time()

## collect all game urls
game_urls = []
driver = webdriver.Chrome(executable_path='/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/Untitled Folder/chromedriver')

driver.get("https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3853/England-Premier-League")

page = Soup(driver.page_source, features='html.parser')
game_urls = []
season_list = [str(2000+i) + "/" + str(2000+i+1) for i in range(22)]

## for all season
for season in tqdm(season_list):
    select = Select(driver.find_element_by_id('seasons'))

    select.select_by_visible_text(season)





    ### crawl all game url of season
    while True:
        page = Soup(driver.page_source, features='html.parser')

        table = page.find_all("a", 'stacked-match-link result-1')
        for i in table:
            game_urls.append(i.get("href"))
        time.sleep(2)
        try:
            button = driver.find_element_by_css_selector("div[class='option-datepicker'] a[class='previous button ui-state-default rc-l is-default']")
            button.click()


        except:
            break

print("Completed!!")
print("Time consumed : ", time.time() - start_time)


100%|███████████████████████████████████████████| 22/22 [31:30<00:00, 85.95s/it]

Completed!!
Time consumed :  1911.4299278259277


In [106]:
len(game_urls)

8135

In [105]:
textfile = open("game_urls.txt", "w")
for element in game_urls:
    textfile.write(element + "\n")
textfile.close()

In [506]:
with open("game_urls.txt", "r") as f:
    game_urls = f.read()

game_urls = game_urls.split("\n")

game_urls = game_urls[:-1]

In [793]:
url.split("-")[3]

'2000'

In [795]:
start_time = time.time()

driver = webdriver.Chrome(executable_path='/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/Untitled Folder/chromedriver')

# driver.get("https://www.whoscored.com/Matches/34119/Show/England-Premier-League-2001-2002-Chelsea-Aston-Villa")
with open("games_whoscored.jl", "w") as f:
    for url in tqdm(game_urls):
        if int(url.split("-")[3]) > 2009:
            res_dict = dict()
        # driver.get("https://www.whoscored.com/Matches/1549539/Preview/England-Premier-League-2021-2022-Brentford-Arsenal")
            driver.get("https://www.whoscored.com" + url)
            try:
                driver.find_element_by_xpath('//a[text()="Preview"]').click()
                time.sleep(2)
                page = Soup(driver.page_source, features='html.parser')

                res_dict = dict()
                res_dict["game_url"] ="https://www.whoscored.com" + url
                res_dict["season"] = url.split("-")[3] + "/" + url.split("-")[4]



                tmp1 = page.title.text.strip().split("-")
                home_team = tmp1[0][:-2]
                away_team = tmp1[1][2:]
                season = tmp1[2].split(" ")[3]
                res_dict["home_team"] = home_team
                res_dict["away_tema"] = away_team
                res_dict["score"] = score
                res_dict['season'] = season

                tmp = page.find_all("div", "info-block cleared")[-1]
                tmp = tmp.find_all("dd")
                tmp = [i.text for i in tmp]

                kickoff = tmp[0]
                date = tmp[1]
                res_dict["time"] = kickoff
                res_dict["date"] = date

                tmp = page.find_all("span", "formation-label")
                tmp = [i.text for i in tmp]
                home_team_formation = tmp[0]
                away_team_formation = tmp[1]
                res_dict["home_team_formation"] = home_team_formation
                res_dict["away_team_formation"] = away_team_formation   
            except:
                tmp = page.find("div", "teams-score-info").find_all("span")
                tmp = [i.text for i in tmp]
                home_team = tmp[0]
                away_team = tmp[4]
                score = tmp[2]
                res_dict["home_team"] = home_team
                res_dict["away_tema"] = away_team
                res_dict["score"] = score

                tmp = page.find_all("div", "info-block cleared")[-1]
                tmp = tmp.find_all("dd")
                tmp = [i.text for i in tmp]

                kickoff = tmp[0]
                date = tmp[1]
                res_dict["time"] = kickoff
                res_dict["date"] = date
                try:
                    tmp = page.find_all("span", "formation-label")
                    tmp = [i.text for i in tmp]
                    home_team_formation = tmp[0]
                    away_team_formation = tmp[1]
                    res_dict["home_team_formation"] = home_team_formation
                    res_dict["away_team_formation"] = away_team_formation
                except:
                    a = 1

            try:
                driver.find_element_by_xpath('//a[text()="Head to Head"]').click()
            except:
                continue
            try:
                driver.find_element_by_xpath('//a[text()="Player Statistics"]').click()
            except:
                continue
            time.sleep(2)
            page = Soup(driver.page_source, features='html.parser')
            home_team_players = json.loads(str(page).split("homePlayers")[1].split("awayPlayers")[0][14:-16])
            away_team_players = json.loads(str(page).split("homePlayers")[1].split("awayPlayers")[1].split("\')\n        };\n")[0][14:])

            res_dict["home_team_players"] = home_team_players
            res_dict["away_team_players"] = away_team_players
            f.write(json.dumps(res_dict))



100%|████████████████████████████████████| 8135/8135 [13:25:35<00:00,  5.94s/it]
